<a href="https://colab.research.google.com/github/qwasd34/DEV_DATA/blob/main/%5B%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%A7%88%EC%9D%B4%EB%8B%9D%5D_%E1%84%83%E1%85%A1%E1%86%AB%E1%84%8B%E1%85%A5_%E1%84%8B%E1%85%B5%E1%86%B7%E1%84%87%E1%85%A6%E1%84%83%E1%85%B5%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 원핫 인코딩 (One-hot Encoding) 단어에 적용하기

In [1]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
sentence = "사과는 맛있다 바나나는 맛있다"

In [3]:
# 문장을 단어로 분리
# 띄어쓰기 단위를 사용
words = sentence.split()
print(words)

['사과는', '맛있다', '바나나는', '맛있다']


In [4]:
# 단어 배열을 NumPy 배열로 변환
words_array = np.array(words).reshape(-1, 1)
print(words_array)

[['사과는']
 ['맛있다']
 ['바나나는']
 ['맛있다']]


In [5]:
# OneHotEncoder 객체 생성
encoder = OneHotEncoder(sparse_output=False) # sparse_output=False : 수업에서 다룬 0과 1로 이루어진 행렬의 형태로 변환

In [6]:
# 원핫 인코딩 적용!
one_hot_encoded = encoder.fit_transform(words_array)
print(one_hot_encoded)

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [7]:
encoder.get_feature_names_out() # 각 열이 어떤 단어를 나타내는지를 표시

array(['x0_맛있다', 'x0_바나나는', 'x0_사과는'], dtype=object)

In [8]:
def one_hot_encode_words(sentence):
    words = sentence.split()
    words_array = np.array(words).reshape(-1, 1)

    encoder = OneHotEncoder(sparse_output=False)
    one_hot_encoded = encoder.fit_transform(words_array)

    return one_hot_encoded, encoder.get_feature_names_out()

In [9]:
example_sentence = '오늘 날씨가 참 맑습니다.'

In [10]:
encoded_words, feature_names = one_hot_encode_words(example_sentence)

In [11]:
encoded_words, feature_names

(array([[0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]]),
 array(['x0_날씨가', 'x0_맑습니다.', 'x0_오늘', 'x0_참'], dtype=object))

# Word2Vec을 단어에 적용하기

In [12]:
from gensim.models import KeyedVectors
from gensim.downloader import load

def use_word2vec(word, model):
    try:
        word_vector = model[word]
        return word_vector
    except KeyError:
        return "단어가 모델의 어휘에 없습니다."

In [13]:
# 제공하는 기본 Word2Vec 모델 불러오기
# 약 10~12분 정도 소요
word2vec_model = load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [14]:
print(use_word2vec('apple', word2vec_model))
print(use_word2vec('instagram', word2vec_model)) # 학습 당시에 존재하지 않던 단어는 임베딩 불가

[-0.06445312 -0.16015625 -0.01208496  0.13476562 -0.22949219  0.16210938
  0.3046875  -0.1796875  -0.12109375  0.25390625 -0.01428223 -0.06396484
 -0.08056641 -0.05688477 -0.19628906  0.2890625  -0.05151367  0.14257812
 -0.10498047 -0.04736328 -0.34765625  0.35742188  0.265625    0.00188446
 -0.01586914  0.00195312 -0.35546875  0.22167969  0.05761719  0.15917969
  0.08691406 -0.0267334  -0.04785156  0.23925781 -0.05981445  0.0378418
  0.17382812 -0.41796875  0.2890625   0.32617188  0.02429199 -0.01647949
 -0.06494141 -0.08886719  0.07666016 -0.15136719  0.05249023 -0.04199219
 -0.05419922  0.00108337 -0.20117188  0.12304688  0.09228516  0.10449219
 -0.00408936 -0.04199219  0.01409912 -0.02111816 -0.13476562 -0.24316406
  0.16015625 -0.06689453 -0.08984375 -0.07177734 -0.00595093 -0.00482178
 -0.00089264 -0.30664062 -0.0625      0.07958984 -0.00909424 -0.04492188
  0.09960938 -0.33398438 -0.3984375   0.05541992 -0.06689453 -0.04467773
  0.11767578 -0.13964844 -0.26367188  0.17480469 -0.

In [15]:
from scipy.spatial.distance import cosine

def word_similarity(word1, word2, model):
    try:
        vector1 = model[word1]
        vector2 = model[word2]

        similarity = 1 - cosine(vector1, vector2) # 코사인 유사도 계산
        return similarity
    except KeyError as e:
        return str(e)

def most_similar(word, model, topn=5):
    try:
        similar_words = model.most_similar(word, topn=topn) # 가장 유사한 단어 찾기
        return similar_words
    except KeyError as e:
        return str(e)

In [16]:
# 두 단어 사이의 유사도 확인
print('football & basketball 유사도 : ' , word_similarity('football', 'basketball', word2vec_model))
print('football & airplane 유사도 : ' , word_similarity('football', 'airplane', word2vec_model))

football & basketball 유사도 :  0.668246865272522
football & airplane 유사도 :  0.1512438803911209


In [17]:
# 특정 단어와 가장 유사한 단어 보여주기
print('football과 가장 유사한 단어 5개는 : ' , most_similar('football', word2vec_model, topn=5))

football과 가장 유사한 단어 5개는 :  [('soccer', 0.731354832649231), ('fooball', 0.7139959335327148), ('Football', 0.7124834060668945), ('basketball', 0.668246865272522), ('footbal', 0.6649289727210999)]


# GloVe를 단어에 적용하기

In [19]:
def use_glove(word, model):
    try:
        word_vector = model[word]
        return word_vector
    except KeyError:
        return "단어가 모델의 어휘에 없습니다."

In [18]:
# 제공하는 GloVe 모델 불러오기
# 약 5분정도 소요
glove_model = load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [20]:
print(use_glove('apple', glove_model))
print(use_glove('airpods', glove_model)) # 역시 학습 당시에 없던 단어는 임베딩 불가

[-0.20842   -0.019668   0.063981  -0.71403   -0.21181   -0.59283
 -0.15316    0.044217   0.63289   -0.84821   -0.21129   -0.19763
  0.19029   -0.56226    0.27126    0.23782   -0.5189    -0.24518
  0.035243   0.096833   0.24898    0.71279    0.038279  -0.10514
 -0.4779    -0.39515   -0.27194   -0.44428    0.06113   -0.2318
 -0.35901   -0.18239    0.035507  -0.087719  -1.0816    -0.42521
  0.003224  -0.45991   -0.043462  -0.39031    0.519      0.21139
 -0.25527    1.1805    -0.19041   -0.12156    0.034186  -0.062316
  0.14421   -0.53366    0.47425   -0.4471     0.58047    0.43578
  0.1321    -0.095712  -0.37182   -0.013837   0.20601   -0.10099
  0.10685   -0.33723    0.10986    0.34796   -0.099839   0.36942
 -0.52917    0.12407   -0.46127   -0.38483   -0.10114   -0.17634
  0.37574    0.16377   -0.2198    -0.26841    0.84706   -0.35619
 -0.083992  -0.20276   -0.56542    0.19112   -0.14134   -0.7812
  0.69188   -0.083628  -0.54293    0.16437    0.037606  -0.68896
 -0.68711   -0.13367   -0.

In [21]:
# 두 단어 사이의 유사도 확인 (GloVe)
print('football & basketball 유사도 : ' , word_similarity('football', 'basketball', glove_model))
print('football & airplane 유사도 : ' , word_similarity('football', 'airplane', glove_model))

football & basketball 유사도 :  0.7341024279594421
football & airplane 유사도 :  0.002223522402346134


In [22]:
# 특정 단어와 가장 유사한 단어 보여주기  (GloVe)
print('football과 가장 유사한 단어 5개는 : ' , most_similar('football', glove_model, topn=5))

football과 가장 유사한 단어 5개는 :  [('soccer', 0.7682591676712036), ('basketball', 0.7341024279594421), ('league', 0.6599653959274292), ('baseball', 0.6479504704475403), ('rugby', 0.6429778933525085)]
